In [1]:
import sys
sys.path.append('../..')

import torch
import torch.nn as nn
from models.mvit.model import CustomMvitModel

%load_ext autoreload
%autoreload 2

In [32]:
device = 'cpu'
torch_dtype = torch.float32
tensor_shape = [1, 3, 32, 224, 224]

model_kwargs = {
    'backbone': 'x2',
    'mvit_kwargs':{
        'num_classes': 1001,
        'backbone_channels': 64,
        'head_channels': 768,
        'arch': {
            'embed_dims': 96,
            'num_layers': 12,
            'num_heads': 1,
            'downscale_indices': [1, 3, 11]
        },
    }
}
model = CustomMvitModel(**model_kwargs).eval()

model = model.to(device=device, dtype=torch_dtype)
with torch.no_grad():
    batch = torch.rand(*tensor_shape).to(device=device, dtype=torch_dtype)
    output = model(batch)

torch.save({'state_dict': model.state_dict()}, 'test.ckpt')
output.shape

torch.Size([1, 1001])

In [2]:
ckpt = torch.load('../../pretrain_weights/efficientx3d-finetuned-0.686.ckpt', map_location='cpu')
for name in ckpt['state_dict']:
    print(name)
    break

model.s1.pathway0_stem_conv_xy.kernel.conv.weight


In [3]:
state_dict = {}
m_dict = ckpt['state_dict']
for name in m_dict:
#     if name == 'projection.weight':
#         state_dict['projection.model.weight'] = m_dict[name]
#         print('1')
#     elif name == 'projection.bias':
#         state_dict['projection.model.bias'] = m_dict[name]
#         print('2')
#     else:
#         state_dict[name] = m_dict[name]
    state_dict[name[6:]] = m_dict[name]
    
torch.save({
    'state_dict': state_dict,
}, '../../pretrain_weights/efficientx3d-finetuned-0.686.ckpt')

In [7]:
state_dict.keys()

dict_keys(['s1.pathway0_stem_conv_xy.kernel.conv.weight', 's1.pathway0_stem_conv.kernel.conv.weight', 's1.pathway0_stem_conv.kernel.bn.weight', 's1.pathway0_stem_conv.kernel.bn.bias', 's1.pathway0_stem_conv.kernel.bn.running_mean', 's1.pathway0_stem_conv.kernel.bn.running_var', 's1.pathway0_stem_conv.kernel.bn.num_batches_tracked', 's2.pathway0_res0._res_proj.kernel.conv.weight', 's2.pathway0_res0._res_proj.kernel.bn.weight', 's2.pathway0_res0._res_proj.kernel.bn.bias', 's2.pathway0_res0._res_proj.kernel.bn.running_mean', 's2.pathway0_res0._res_proj.kernel.bn.running_var', 's2.pathway0_res0._res_proj.kernel.bn.num_batches_tracked', 's2.pathway0_res0.layers.conv_0.kernel.conv.weight', 's2.pathway0_res0.layers.conv_0.kernel.bn.weight', 's2.pathway0_res0.layers.conv_0.kernel.bn.bias', 's2.pathway0_res0.layers.conv_0.kernel.bn.running_mean', 's2.pathway0_res0.layers.conv_0.kernel.bn.running_var', 's2.pathway0_res0.layers.conv_0.kernel.bn.num_batches_tracked', 's2.pathway0_res0.layers.conv_

In [5]:
from models.mobilenetv2 import MobileNetV2

In [2]:
ckpt = torch.load('./submit/distilated-mobilenet-32.ckpt', map_location='cpu')

In [8]:
model_kwargs = {
    'num_classes': 1001,
    'sample_size': 224,
    'frame_count': 32,
    'temporal_indices': [1, 2, 3, 4, 5, 6],
    'width_mult': 4,
    'with_two_heads': False,
}
model = MobileNetV2(**model_kwargs)
m, u = model.load_state_dict(ckpt['state_dict'], strict=True)
print(f'missing: {m} | unexpected: {u}')
mdoel = model.to(dtype=torch.float16)

missing: [] | unexpected: []


In [9]:
torch.save({
    'state_dict': model.state_dict(),
}, './submit/distilated-mobilenet-16.ckpt')

In [15]:
import pandas as pd


df1 = pd.read_csv('/home/user/Projects/denk_baseline/projects/aij_2023/data/df_train.csv')
df2 = pd.read_csv('/home/user/Projects/denk_baseline/projects/aij_2023/data/df_test.csv')

df3 = pd.concat([df1, df2])
df1.shape, df2.shape, df3.shape

((16160, 4), (2020, 4), (18180, 4))

In [17]:
df1

,attachment_id,label,begin,end
0,7750186b-035d-4c85-8c81-01096d93046c,1000,0,40
1,c1a3317e-e57e-4877-8be0-de592a911a60,1000,0,47
2,4a630113-7051-4069-8c58-d0edf956b046,584,25,71
3,89d65491-6edb-415d-93c3-efca6007a083,627,54,89
4,1a317df7-0b41-453b-afe1-175afe7ecb19,437,12,31
...,...,...,...,...
16155,2116fbba-63ba-477b-a454-24dcfd535d22,291,29,100
16156,9809c94f-455d-448a-9e3e-f9b2c8b1739f,296,19,96
16157,27baae81-6ddc-48f0-972a-577c1ceac366,260,19,64
16158,e4edb4bf-97d0-425e-a208-39fb3aaabe38,269,22,76


In [16]:
df3

,attachment_id,label,begin,end
0,7750186b-035d-4c85-8c81-01096d93046c,1000,0,40
1,c1a3317e-e57e-4877-8be0-de592a911a60,1000,0,47
2,4a630113-7051-4069-8c58-d0edf956b046,584,25,71
3,89d65491-6edb-415d-93c3-efca6007a083,627,54,89
4,1a317df7-0b41-453b-afe1-175afe7ecb19,437,12,31
...,...,...,...,...
2015,c87c8166-3e29-4cb3-8f6a-7641b2102af7,168,14,56
2016,794d8fde-e6d9-4986-9529-b6dfdb97510c,775,21,71
2017,250c5513-8e5f-4549-9ecd-4b108af0fe86,165,19,67
2018,e8c4a21b-dce1-483a-9915-577fc794e2c0,592,11,89
